In [1]:
import os
import re
import yaml
import pickle

import ipyplot
import pandas as pd


In [2]:
timelines_path = '../../reports/timelines/colour timelines/'
timelines_notebook = 'http://localhost:8888/view/notebooks/8%20-%20timelines/colour%20timelines/'
addresses = os.listdir(timelines_path)
addresses = [ad for ad in addresses if 'png' in ad]
local_addresses = [(timelines_path + addr) for addr in addresses]

In [3]:
with open('../../data/post_test/rankings.pkl', 'rb') as fp:
    rankings = pickle.load(fp)
    id_rankings = {rankings.iloc[i]['username']: rankings.iloc[i]['ranking'] for i in range(len(rankings))}

In [4]:
def get_map(label_map:str) -> dict:
    if label_map == 'vector_labels':
        map_path = '../../data/experiment_keys/permutation_maps/vector_binary.yaml'
        
    with open(map_path) as fp:
        return yaml.load(fp, Loader=yaml.FullLoader)
    
def images_labels(addresses:list, id_rankings:dict, label_map:dict):
    permutation = re.compile('^p([0-9]+)_')
    learner_id = re.compile('_l([A-z0-9]+)_t')
    task = re.compile('_t([0-9]+).')
    
    permutations = []
    learners = []
    labels = []
    tasks = []
    for address in addresses:
        try:
            p = permutation.findall(address)[0]
            permutations.append(p)

            l = learner_id.findall(address)[0]
            learners.append(l)

            labels.append(label_map['map'][p])

            t = task.findall(address)[0]
            tasks.append(t)
        except IndexError:
            permutations.append('missing')

            l = learner_id.findall(address)[0]
            learners.append(l)

            labels.append('none')

            t = task.findall(address)[0]
            tasks.append(t)
        
    return permutations, learners, labels, tasks

def plot_for_task(addresses:list, labels:list, tasks:list, task:int):
    indices = [i for i in range(len(tasks)) if tasks[i] == str(task)]
    ad = [addresses[idx] for idx in indices]
    labs = [labels[idx] for idx in indices]
    ipyplot.plot_class_tabs(ad, labs, max_imgs_per_tab=30, img_width=700)
    
    

In [5]:
label_map = get_map('vector_labels')
permutations, learners, labels, tasks = images_labels(addresses, id_rankings, label_map)
plot_for_task(local_addresses, labels, tasks, 2)